In [1]:
!pip install flickrapi

In [2]:
import flickrapi


import logging
import functools
import json

import urllib.request
from PIL import Image

import os
import pathlib

In [3]:
# Flickr api access key 
flickr=flickrapi.FlickrAPI('e358383789f8d18e68d4fa0bc1990e47', '1c28595375ef2fce', cache=True)

In [4]:
def safe_name(filename):
    """
    Generate a safe file name from a string
    """
#     return "".join([c for c in filename if c.isalpha() or c.isdigit() or c==' ']).rstrip()
    return filename.replace('/', '')

In [5]:
def max_resolution(labels):
    """
    Pick the best resolution from available resolutions:
    Choose from best options like:
    ['Square', 'Large Square', 'Thumbnail', 'Small', 'Small 320', 'Small 400', 'Medium', 'Medium 640', 'Medium 800', 'Large', 'Large 1600', 'Large 2048']
    """
    original_label = 'Original'
    if original_label in labels:
        return original_label
    
    def sort_by_size(labels, size='Large'):
        labels_by_size = filter(lambda label: size in label and any(char.isdigit() for char in label), labels)
        # Extract dimension (second string in label) for sorting
        labels_by_size = sorted(labels_by_size, key=lambda label: int(label.split(" ")[1]), reverse=True)
        return labels_by_size
    
    large = sort_by_size(labels, 'Large')
    medium = sort_by_size(labels, 'Medium')
    small = sort_by_size(labels, 'Small')
    
    best_to_worst_resolution = large + medium + small
    return best_to_worst_resolution[0]

In [6]:
@functools.lru_cache()
def get_nsid(**kwargs):
    """
    lookup user NSID by username. NSID is required for other API operations
    under the people submodule.
    """
    if 'username' not in kwargs:
        raise Exception(f'expected `username` in kwargs')
    username = kwargs['username']
    response = json.loads(
        flickr.people.findByUsername(username=username, format='json')
    )
    
    # In case of API failure, such as {'stat': 'fail', 'code': 1, 'message': 'User not found'}
    if 'code' in response and response['code'] == 1:
        raise Exception(response)
    
    return response['user']['nsid']

In [7]:
@functools.lru_cache()
def download_photos(
    username_or_id=None, 
    limit=100):
    """
    List all of the photo urls for a user.
    :param username_or_id: either the screen username (if unique) or the user ID from the URL.
    :param limit:    the number of photos to get
    :param label:    the label of photos to get. e.g: 'Square', 'Large Square', 'Thumbnail', 'Small', 'Small 320', 'Small 400', 'Medium', 'Medium 640', 'Medium 800', 'Large', 'Large 1600', 'Original'
    
    """
    
    if username_or_id is  None:
        raise Exception('username_or_id must be supplied')

    if limit < 1:
        raise Exception('limit must be greater than 1')
    
    try:
        user_id = get_nsid(username=username_or_id),
        
    except Exception as e:
        user_id = username_or_id
        
    
    # fetch name info
    try:
        response = json.loads(flickr.people.getInfo(
                user_id=user_id,
                format='json'
            ))
    except Exception as e:
        print(f'failed flickr.people.getInfo { {user_id: user_id} }')
        print(e)
        return
        
    
    # if there is a realname and username that differ
    if 'realname' in response['person'] and response['person']['realname']['_content'] != response['person']['username']['_content']:
        name = f"{response['person']['username']['_content']} ({response['person']['realname']['_content']})"
    else:
        name = f"{response['person']['username']['_content']}"
    
    print(name)
    name = safe_name(name)

    
    # make a folder to save output images
    output_folder = f'images/{name}'
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    try:
        response = json.loads(flickr.people.getPhotos(
            user_id=user_id,
            format='json'
        ))
            
    except Exception as e:
        print(f'failed initially flickr.people.getPhoto { {user_id: user_id} }')
        raise e

    if 'photos' not in response:
        raise Exception(response)
    num_pages = response['photos']['pages']
    
    # current number of photos, if starting from a failed or interrupted run
    total_photos = len(os.listdir(output_folder))
    
    for page in range(1, num_pages+1):
        try:
            response = json.loads(flickr.people.getPhotos(
                user_id=user_id,
                page=page,
                format='json'
            ))
        except Exception as e:
            print(f'failed flickr.people.getPhoto { {user_id: user_id, page: page} }')
            print(e)
            continue
        
        photos = response['photos']['photo']
        
        for photo in response['photos']['photo']:
            title = photo['title']
            photo_id = photo['id']
            
            # for extra long titles, truncate 
            filename = f'{output_folder}/{safe_name(title)}'[:255-4] + '.jpg'
            
            # already downloaded
            if os.path.exists(filename):
                continue
            
            # get all photos
            max_resolution_size = None
            try:
                response = json.loads(flickr.photos.getSizes(photo_id=photo_id, format='json'))
                sizes = response['sizes']['size']
                size_labels = [size['label'] for size in sizes]
                max_resolution_label = max_resolution(size_labels)    

                max_resolution_sizes = [size for size in sizes if size['label'] == max_resolution_label]
                max_resolution_size = max_resolution_sizes[0]
            except Exception as e:
                print(f'failed to load photo { photo_id }')
                print(e)
                continue
            
            url = None
            try:
                # save image from url to local under images/[username]/[title].jpg
                with open(filename, 'wb') as f:
                    url = max_resolution_size['source']
                    response = urllib.request.urlopen(url)
                    f.write(response.read())
            except Exception as e:
                print(f'failed to save photo {url}')
                print(e)
                continue
                
                
            # progress
            
            total_photos += 1

            print(f'({total_photos}/{min(len(photos), limit)})\t{filename}')
            
            if total_photos >= limit:
                print('\n')
                return

In [8]:
# Load people you are following
response = json.loads(
    flickr.contacts.getPublicList(
        user_id=get_nsid(username='alex_mocs'),
        format='json'
    ))

nsids = map( lambda x: x['nsid'], response['contacts']['contact'])

In [9]:
for nsid in nsids:
    try: 
        download_photos(username_or_id=nsid, limit=50)
    except Exception as e:
        print(f"failed download_photos { {'username_or_id': nsid} }")
        print(e)

!ggs (Iggs)
-Disty-
1saac W.
[Jack Frost]
[Oblivious]
[Rhymes_Shelter] (Vlad Lisin)
[VB]
_CZQ_ (Mitch  Henry)
A Plastic Infinity (Aaron Van Cleave)
failed to load photo 10674360615
argument of type 'int' is not iterable
(316/50)	images/A Plastic Infinity (Aaron Van Cleave)/The Great Pit of Carkoon.jpg


Aardvark17_
Aiden.Builds
AjRed17
AlexParkDesigns
anderson_builder (Benjamin Anderson)
Andrea Lattanzio (Norton74)
Anthony (The Secret Walrus) Wilson (Anthony Wilson)
awesomenessborn
Azurekingfisher
failed to load photo 24743943135
argument of type 'int' is not iterable
failed to load photo 24062595253
argument of type 'int' is not iterable
(303/50)	images/Azurekingfisher/station-sketch-02.jpg


Ballom Nom Nom (Matt Goldberg)
(303/50)	images/Ballom Nom Nom (Matt Goldberg)/Tarakava.jpg


BardJaskier
(303/50)	images/BardJaskier/Battlesuit Archangel #1.jpg


Bart De Dobbelaer
(303/50)	images/Bart De Dobbelaer/Arachnophobia.jpg


Belicure
Ben Cossy
(225/50)	images/Ben Cossy/Master and Appren

failed to load photo 50026338592
invalid literal for int() with base 10: '3K'
failed to load photo 50025533928
invalid literal for int() with base 10: '3K'
failed to load photo 49991159102
invalid literal for int() with base 10: '3K'
failed to load photo 49990392633
invalid literal for int() with base 10: '3K'
failed to load photo 49990917076
invalid literal for int() with base 10: '3K'
failed to load photo 49991159202
invalid literal for int() with base 10: '3K'
failed to load photo 49991159227
invalid literal for int() with base 10: '3K'
failed to load photo 49990173818
invalid literal for int() with base 10: '3K'
failed to load photo 49990698206
invalid literal for int() with base 10: '3K'
failed to load photo 49990698221
invalid literal for int() with base 10: '3K'
failed to load photo 49990940272
invalid literal for int() with base 10: '3K'
failed to load photo 49990940292
invalid literal for int() with base 10: '3K'
failed to load photo 49990698326
invalid literal for int() with 

failed to load photo 49090007346
invalid literal for int() with base 10: '3K'
failed to load photo 49077054473
invalid literal for int() with base 10: '3K'
failed to load photo 49077584011
invalid literal for int() with base 10: '3K'
failed to load photo 49077584106
invalid literal for int() with base 10: '3K'
failed to load photo 49044010697
invalid literal for int() with base 10: '3K'
failed to load photo 49043798276
invalid literal for int() with base 10: '3K'
failed to load photo 49044011822
invalid literal for int() with base 10: '3K'
failed to load photo 49015673283
invalid literal for int() with base 10: '3K'
failed to load photo 49016193056
invalid literal for int() with base 10: '3K'
failed to load photo 49007581927
invalid literal for int() with base 10: '3K'
failed to load photo 49006835708
invalid literal for int() with base 10: '3K'
failed to load photo 49007582047
invalid literal for int() with base 10: '3K'
failed to load photo 49007367966
invalid literal for int() with 

failed to load photo 48095733823
invalid literal for int() with base 10: '3K'
failed to load photo 48030738423
invalid literal for int() with base 10: '3K'
failed to load photo 48030803867
invalid literal for int() with base 10: '3K'
failed to load photo 48030803887
invalid literal for int() with base 10: '3K'
failed to load photo 48030738833
invalid literal for int() with base 10: '3K'
failed to load photo 48019268047
invalid literal for int() with base 10: '3K'
failed to load photo 48019189968
invalid literal for int() with base 10: '3K'
failed to load photo 48019268167
invalid literal for int() with base 10: '3K'
failed to load photo 48019268252
invalid literal for int() with base 10: '3K'
failed to load photo 47982768733
invalid literal for int() with base 10: '3K'
failed to load photo 47982817816
invalid literal for int() with base 10: '3K'
failed to load photo 40942971563
invalid literal for int() with base 10: '3K'
failed to load photo 47120269044
invalid literal for int() with 

failed to load photo 33228500978
invalid literal for int() with base 10: '3K'
failed to load photo 40091943823
invalid literal for int() with base 10: '3K'
failed to load photo 46143205245
invalid literal for int() with base 10: '3K'
failed to load photo 32115213417
invalid literal for int() with base 10: '3K'
failed to load photo 32115218327
invalid literal for int() with base 10: '3K'
failed to load photo 33174140918
invalid literal for int() with base 10: '3K'
failed to load photo 47038982951
invalid literal for int() with base 10: '3K'
failed to load photo 46125384435
invalid literal for int() with base 10: '3K'
failed to load photo 40074194043
invalid literal for int() with base 10: '3K'
failed to load photo 46315093184
invalid literal for int() with base 10: '3K'
failed to load photo 46120660535
invalid literal for int() with base 10: '3K'
failed to load photo 33159145858
invalid literal for int() with base 10: '3K'
failed to load photo 47034333161
invalid literal for int() with 

...

(39/50)	images/theunderscoreddouble1 (The Underscored Double)/Odd Noises in Empty Rooms: Deep Skeleton.jpg
(40/50)	images/theunderscoreddouble1 (The Underscored Double)/Together Through Time.jpg
(41/50)	images/theunderscoreddouble1 (The Underscored Double)/Helryx, the First Toa.jpg
(42/50)	images/theunderscoreddouble1 (The Underscored Double)/Sister of the Skrall.jpg
(43/50)	images/theunderscoreddouble1 (The Underscored Double)/The Monkey.jpg
(44/50)	images/theunderscoreddouble1 (The Underscored Double)/Cuttlefish.jpg
(45/50)	images/theunderscoreddouble1 (The Underscored Double)/Sub-Zero Scorpio.jpg
(46/50)	images/theunderscoreddouble1 (The Underscored Double)/Hermit’s Reef.jpg
(47/50)	images/theunderscoreddouble1 (The Underscored Double)/The Brazoraptor.jpg
(48/50)	images/theunderscoreddouble1 (The Underscored Double)/The Afterman: Charlie D..jpg
(49/50)	images/theunderscoreddouble1 (The Underscored Double)/PITtober 2019: Vent Crawler.jpg
(50/50)	images/theunderscoreddouble1 (The Unde

(31/50)	images/Tino Poutiainen/Glass Cerberus.jpg
(32/50)	images/Tino Poutiainen/Serpent`s strait.jpg
(33/50)	images/Tino Poutiainen/Archangel Michael.jpg
(34/50)	images/Tino Poutiainen/Mictlantecuhtli.jpg
(35/50)	images/Tino Poutiainen/Crabvanced diving suit.jpg
(36/50)	images/Tino Poutiainen/Birchwood elk.jpg
(37/50)	images/Tino Poutiainen/[The Hivemind].jpg
(38/50)	images/Tino Poutiainen/Death and rebirth.jpg
(39/50)	images/Tino Poutiainen/Mustafar mining facility.jpg
(40/50)	images/Tino Poutiainen/C-wing fleet barrier link.jpg
(41/50)	images/Tino Poutiainen/Aiila-class battleram.jpg
(42/50)	images/Tino Poutiainen/Spire of the Neoprophets.jpg
(43/50)	images/Tino Poutiainen/Evangelion Unit 02.jpg
(44/50)	images/Tino Poutiainen/Ironclad: K02 "Mastodon".jpg
(45/50)	images/Tino Poutiainen/Risk of Rain 2.jpg
(46/50)	images/Tino Poutiainen/Type-G bipedal ambulance.jpg
(47/50)	images/Tino Poutiainen/Grand Admiral.jpg
(48/50)	images/Tino Poutiainen/Where`s your mother, little one?.jpg
(49/5

(13/35)	images/Umakri ()/Sierra wip.jpg
(14/35)	images/Umakri ()/wip eva.jpg
(15/35)	images/Umakri ()/my solution for parts non existent before 2017; mega bloks.jpg
VelociJACKtor
(1/50)	images/VelociJACKtor/2022.jpg
(2/50)	images/VelociJACKtor/Cable’s Fate.jpg
(3/50)	images/VelociJACKtor/PITtober 2022: Pterygotor.jpg
(4/50)	images/VelociJACKtor/Crocodile Tears.jpg
(5/50)	images/VelociJACKtor/Bota-Toro.jpg
(6/50)	images/VelociJACKtor/Warden of the Nine Circles.jpg
(7/50)	images/VelociJACKtor/Arthropodal Annihilation.jpg
(8/50)	images/VelociJACKtor/Hunter of Hell Creek.jpg
(9/50)	images/VelociJACKtor/Warhammer Titan.jpg
(10/50)	images/VelociJACKtor/Jaw Titan.jpg
(11/50)	images/VelociJACKtor/The Rumbling.jpg
(12/50)	images/VelociJACKtor/Grab.jpg
(13/50)	images/VelociJACKtor/Toe Biter.jpg
(14/50)	images/VelociJACKtor/Toa Tuyet.jpg
(15/50)	images/VelociJACKtor/2021.jpg
(16/50)	images/VelociJACKtor/Shin Godzilla.jpg
(17/50)	images/VelociJACKtor/Wanderers of the Void.jpg
(18/50)	images/Veloci

(44/50)	images/woomy world (Woomy World)/IMG_20191211_074801_760.jpg
(45/50)	images/woomy world (Woomy World)/Green Ninja Mech Dragon (Kaiju Edition).jpg
(46/50)	images/woomy world (Woomy World)/Ice Sorceress.jpg
(47/50)	images/woomy world (Woomy World)/Super Space Palace.jpg
(48/50)	images/woomy world (Woomy World)/The Phantom Menace Diorama.jpg
(49/50)	images/woomy world (Woomy World)/Winter.jpg
(50/50)	images/woomy world (Woomy World)/Aerial View.jpg


yannickdesignsthings
(1/50)	images/yannickdesignsthings/Poro Coco.jpg
(2/50)	images/yannickdesignsthings/Spiky 3.jpg
(3/50)	images/yannickdesignsthings/Spiky 2.jpg
(4/50)	images/yannickdesignsthings/Off-Road Monster Spiky.jpg
(5/50)	images/yannickdesignsthings/Poro Kali.jpg
(6/50)	images/yannickdesignsthings/The Holy Gunslinger.jpg
(7/50)	images/yannickdesignsthings/The Holy Gunslinger (Speak No Evil).jpg
(8/50)	images/yannickdesignsthings/>HOW DO YOU LIKE YOUR EGGS.jpg
(9/50)	images/yannickdesignsthings/Garuda-Class Starfighter.jpg
(

(15/50)	images/▷Cezium◁/Icarax.jpg
(16/50)	images/▷Cezium◁/CrayStation4.jpg
(17/50)	images/▷Cezium◁/Temperance.jpg
(18/50)	images/▷Cezium◁/Dragonslayer.jpg
(19/50)	images/▷Cezium◁/Luminescent Leviathan.jpg
(20/50)	images/▷Cezium◁/Luminescent Leviathan - Side.jpg
(21/50)	images/▷Cezium◁/Movorak.jpg
(22/50)	images/▷Cezium◁/Ronin.jpg
(23/50)	images/▷Cezium◁/Fairy Fairy.jpg
(24/50)	images/▷Cezium◁/H-301 Autonomous Reconnaisance Unit.jpg
(25/50)	images/▷Cezium◁/Emma.jpg
(26/50)	images/▷Cezium◁/Deathlord Iskandir.jpg
(27/50)	images/▷Cezium◁/Chancellor Adramelech.jpg
(28/50)	images/▷Cezium◁/Plague Mech Ⲭ: Wolf Spider.jpg
(29/50)	images/▷Cezium◁/One small step for crab....jpg
(30/50)	images/▷Cezium◁/GreenGuy.jpg
(31/50)	images/▷Cezium◁/Baba Ludmila.jpg
(32/50)	images/▷Cezium◁/Bowsette.jpg
(33/50)	images/▷Cezium◁/Bowsette - Back.jpg
(34/50)	images/▷Cezium◁/Mikael.jpg
(35/50)	images/▷Cezium◁/GYR-58.jpg
(36/50)	images/▷Cezium◁/Tarakava-Chan.jpg
(37/50)	images/▷Cezium◁/Rahi Musume! v0.03 Pre-Alpha